In [5]:
import yfinance as yf
import pandas as pd
from tilib import indicators as id
import numpy as np
from datetime import datetime, timedelta
from ftplib import FTP

In [4]:
end_date = datetime.now()
start_date = end_date - timedelta(days = 59) # 60 Days is for 5 min data
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')
#start = start_date_str, end = end_date_str

In [17]:
security_df = yf.Ticker('^SPX')
security_df.info
hist = security_df.history(start = start_date_str, end =  end_date_str, interval = '5m') # start = '2023-11-23', end = '2024-01-21'
                                                                 #Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
df = pd.DataFrame(hist[['Volume', 'Close', 'High', 'Low']]).round(2)
df_cumulative_delta = pd.DataFrame(hist[['Volume', 'Close', 'High', 'Low']]).round(2) 
df.head()

,Volume,Close,High,Low
Datetime,,,,
2023-11-24 09:30:00-05:00,0,4558.12,4559.77,4553.34
2023-11-24 09:35:00-05:00,23668067,4555.68,4558.15,4553.81
2023-11-24 09:40:00-05:00,20852297,4554.19,4556.23,4553.40
2023-11-24 09:45:00-05:00,21793962,4554.72,4555.65,4552.80
2023-11-24 09:50:00-05:00,0,4556.46,4557.11,4554.31


In [7]:
#Calcualting HLC3 value
df['HLC3'] = (df['High'] + df['Low'] + df['Close']) / 3

# Calculating VWAP (using HLC3)
df['VWAP'] = (df['HLC3'] * df['Volume']).cumsum() / df['Volume'].cumsum()

#Calculating Standard Deviation 
df['PriceDeviation'] = (df['HLC3'] - df['VWAP']).rolling(window = 78).std() ##Window 78 set based on 5 min trading data (78 5 min in 1 day)##
# Session based (How it was on Pinescript) appears to be based on daily 

# Calculate bands [UpperBand_1,	LowerBand_1, UpperBand_2, LowerBand_2, UpperBand_3, LowerBand_3]
for i in [1, 2, 3]:
    df[f'UpperBand_{i}'] = df['VWAP'] + (i * df['PriceDeviation'])
    df[f'LowerBand_{i}'] = df['VWAP'] - (i * df['PriceDeviation'])

df = df.dropna()
df_hlc3_vwap = df
df_hlc3_vwap

,Volume,Close,High,Low,HLC3,VWAP,PriceDeviation,UpperBand_1,LowerBand_1,UpperBand_2,LowerBand_2,UpperBand_3,LowerBand_3
Datetime,,,,,,,,,,,,,
2023-11-27 12:30:00-05:00,13504235,4555.77,4556.27,4555.08,4555.706667,4554.947027,2.190676,4557.137703,4552.756351,4559.328379,4550.565675,4561.519055,4548.374999
2023-11-27 12:35:00-05:00,14557713,4555.25,4556.61,4554.77,4555.543333,4554.952776,2.195841,4557.148616,4552.756935,4559.344457,4550.561094,4561.540298,4548.365253
2023-11-27 12:40:00-05:00,13000512,4557.76,4557.83,4555.23,4556.940000,4554.969737,2.223961,4557.193698,4552.745775,4559.417659,4550.521814,4561.641620,4548.297853
2023-11-27 12:45:00-05:00,12673889,4557.22,4558.32,4557.11,4557.550000,4554.991029,2.262426,4557.253455,4552.728603,4559.515881,4550.466177,4561.778306,4548.203751
2023-11-27 12:50:00-05:00,15288984,4557.73,4557.90,4556.57,4557.400000,4555.014773,2.284833,4557.299606,4552.729939,4559.584439,4550.445106,4561.869272,4548.160273
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-19 15:35:00-05:00,35700969,4836.70,4839.36,4836.59,4837.550000,4692.754675,16.181672,4708.936346,4676.573003,4725.118018,4660.391331,4741.299689,4644.209660
2024-01-19 15:40:00-05:00,41308093,4834.94,4837.80,4834.94,4835.893333,4692.833385,15.908680,4708.742065,4676.924705,4724.650745,4661.016025,4740.559425,4645.107345
2024-01-19 15:45:00-05:00,51306388,4837.06,4837.66,4834.61,4836.443333,4692.931402,15.656405,4708.587807,4677.274998,4724.244211,4661.618593,4739.900616,4645.962189


In [16]:
ma_df = pd.DataFrame(df['Close'])
ma_df['MA 20'] = ma_df['Close'].rolling(window = 20).mean()
ma_df['MA 50'] = ma_df['Close'].rolling(window = 50).mean()
ma_df = ma_df.dropna()
ma_df

,Close,MA 20,MA 50
Datetime,,,
2023-11-28 10:05:00-05:00,4544.06,4549.4515,4553.3854
2023-11-28 10:10:00-05:00,4544.36,4549.0745,4553.1572
2023-11-28 10:15:00-05:00,4546.66,4548.8145,4552.9854
2023-11-28 10:20:00-05:00,4546.42,4548.5885,4552.7586
2023-11-28 10:25:00-05:00,4546.08,4548.2345,4552.5358
...,...,...,...
2024-01-19 15:35:00-05:00,4836.70,4836.2395,4822.8090
2024-01-19 15:40:00-05:00,4834.94,4836.3125,4823.4704
2024-01-19 15:45:00-05:00,4837.06,4836.4230,4824.2144


In [15]:
import urllib.request as ftp


ftp.request.urlretrieve('ftp://ftp.databento.com/KYWSGKN3/XNAS-20240122-J7M5BLL6L7', 'file')
ftp.login(user='KYWSGKN3', passwd='#NismoZFL3')  # Replace with your username and password

AttributeError: module 'urllib.request' has no attribute 'request'

In [ ]:
# # Calculate Cumulative Delta
# data['BuyVolume'] = np.where(data['Close'] >= data['Open'], data['Volume'], 0)
# data['SellVolume'] = np.where(data['Close'] < data['Open'], data['Volume'], 0)
# data['Delta'] = data['BuyVolume'] - data['SellVolume']
# data['CumulativeDelta'] = data['Delta'].cumsum()

# # Calculate Moving Averages
# maLength1 = 20
# maLength2 = 50
# data['MA_CumDelta_1'] = data['CumulativeDelta'].rolling(window=maLength1).mean()
# data['MA_CumDelta_2'] = data['CumulativeDelta'].rolling(window=maLength2).mean()

# # VWAP calculation
# # Note: VWAP typically resets daily (or on your chosen timeframe)
# data['VWAP'] = (data['Volume'] * (data['High'] + data['Low'] + data['Close']) / 3).cumsum() / data['Volume'].cumsum()

# # Detect Divergences
# # Simplified example - you might need more sophisticated logic
# data['BullishDivergence'] = (data['Close'] > data['Open']) & (data['Delta'] < data['Delta'].shift(1))
# data['BearishDivergence'] = (data['Close'] < data['Open']) & (data['Delta'] > data['Delta'].shift(1))